In [1]:
import numpy as np
import xarray as xr
import os,errno
import sys
import datetime as dt
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd
%matplotlib inline
dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/ghcnd_all/'#edited
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'

In [2]:
if 1==1:#step 1: get stations with 95% completeness from 1979 - 2019 and data from Sep 1979 and Nov 2019
    ds=pd.read_csv(dir1+'ghcnd-stations_new.txt',header=None)
    print(ds)
    count=0
    yr_start=1979
    yr_end=2019
    count=0
    count_ne=0
    stations=[]
    lats=[]
    lons=[]
    elevs=[]
    station_names=[]
    thresholds_annual_99=[]
    thresholds_annual_95=[]
    thresholds_winter_99=[]
    thresholds_spring_99=[]
    thresholds_summer_99=[]
    thresholds_fall_99=[]
    thresholds_winter_95=[]
    thresholds_spring_95=[]
    thresholds_summer_95=[]
    thresholds_fall_95=[]
    for row in ds.iterrows():
        if count%500==0:
            print(count)
        count=count+1
        if row[1][0][0:2]=='US':
            filename=row[1][0][0:11]
            rest_space=row[1][0][11:]
            split=rest_space.split()
            
            lat= float(split[0])
            lon= float(split[1])
            elev=float(split[2])
            state=split[3]
            station_name=split[4]
            
            if station_name in ['MT','NEW'] or station_name[0]==station_name:#just trying to get more unique station names but NOT a complete set; use filenames which are unique
                try:
                    station_name=station_name+' '+split[5]
                except:
                    pass

            station_points=[(-73.8092, 42.7431),(-73.8842, 41.6267),(-75.985, 42.1997),(-74.155, 41.7681),(-74.6142, 41.5917),(-73.9692, 40.7789),(-73.4167, 41.4),(-74.6625, 41.2214),(-75.4492, 40.6508),(-75.7269, 41.3336)]
            names=['']
            if state in ['MD','ME','NH','MA','RI','CT','VT','NY','NJ','PA','VA','WV','DE']:
                filename=dir1+row[1][0][0:11]+'.dly'
                years=[]
                months=[]
                obs_list=[]
                obs_winter=[]
                obs_spring=[]
                obs_summer=[]
                obs_fall=[]
                ds1=pd.read_csv(filename,header=None)#,sep=None)
                nancount=0
                qcount=0
                yescount=0
                for row1 in ds1.iterrows():
                    row_neg_space=row1[1][0].replace('-',' -')
                    row_basic=row1[1][0]
                    a=row_neg_space.split()
                    code=a[0][-4:]
                    if code=='PRCP':
                        
                        year=int(a[0][11:15])
                        month=int(a[0][15:17])
                        if yr_start<=year<=yr_end:
                            
                            years.append(year)
                            months.append(month)
                            if month in [1,3,5,7,8,10,12]:
                                days=31
                            if month in [4,6,9,11]:
                                days=30
                            if month==2 and year%4==0:
                                days=29
                            if month==2 and year%4!=0:
                                days=28
                            for i in range(days):
                                #print(row_basic)
                                mflag=row_basic[26+8*i]
                                qflag=row_basic[27+8*i]
                                sflag=row_basic[28+8*i]
                                obs=float(row_basic[21+8*i:26+8*i])
                                if obs==-9999.0:
                                    nancount=nancount+1
                                elif qflag!=' ':
                                    qcount=qcount+1
                                else:
                                    
                                    yescount=yescount+1
                                    obs_list.append(obs)
                                    
                                    if month in [12,1,2] and 1980<=year<=2018 or month==12 and year==1979 or month in [1,2] and year==2019:
                                        obs_winter.append(obs)
                                    if month in [3,4,5]:
                                        obs_spring.append(obs)
                                    if month in [6,7,8]:
                                        obs_summer.append(obs)
                                    if month in [9,10,11]:
                                        obs_fall.append(obs)
                                    #tuple=(year,month,i,obs)
                                    #print(tuple)
                            #month=int(a[0][-6:-4])
                
                ratio=float(yescount)/float((365*41+10))
                zip_years_months=list(zip(years,months))
                months_yr_start=[x[1] for x in zip_years_months if x[0]==yr_start]
                months_yr_end=[x[1] for x in zip_years_months if x[0]==yr_end]
                months_yr_start=list(set(months_yr_start))
                months_yr_end=list(set(months_yr_end))

            
                if len(years)>0 and ratio>=0.95 and len(months_yr_start)>0 and len(months_yr_end)>0:
                    if min(years)==yr_start and max(years)==yr_end:# and min(months_yr_start)<=9 and max(months_yr_end)==12:
                        
                        measurable=[x for x in obs_list if x>=1]#0.1mm = measurable
                        
                        measurable_winter=[x for x in obs_winter if x>=1]
                        measurable_spring=[x for x in obs_spring if x>=1]
                        measurable_summer=[x for x in obs_summer if x>=1]
                        measurable_fall=[x for x in obs_fall if x>=1]
                        
                        threshold_winter_99=np.percentile(measurable_winter,99,interpolation='linear')
                        threshold_spring_99=np.percentile(measurable_spring,99,interpolation='linear')
                        threshold_summer_99=np.percentile(measurable_summer,99,interpolation='linear')
                        threshold_fall_99=np.percentile(measurable_fall,99,interpolation='linear')
                        
                        threshold_winter_95=np.percentile(measurable_winter,95,interpolation='linear')
                        threshold_spring_95=np.percentile(measurable_spring,95,interpolation='linear')
                        threshold_summer_95=np.percentile(measurable_summer,95,interpolation='linear')
                        threshold_fall_95=np.percentile(measurable_fall,95,interpolation='linear')
                        
                        threshold_annual_99=np.percentile(measurable,99,interpolation='linear')
                        threshold_annual_95=np.percentile(measurable,95,interpolation='linear')
                        thresholds_annual_99.append(threshold_annual_99)
                        thresholds_annual_95.append(threshold_annual_95)
                        
                        thresholds_winter_99.append(threshold_winter_99)
                        thresholds_spring_99.append(threshold_spring_99)
                        thresholds_summer_99.append(threshold_summer_99)
                        thresholds_fall_99.append(threshold_fall_99)
                        
                        thresholds_winter_95.append(threshold_winter_95)
                        thresholds_spring_95.append(threshold_spring_95)
                        thresholds_summer_95.append(threshold_summer_95)
                        thresholds_fall_95.append(threshold_fall_95)
                        
                        stations.append(filename)
                        lats.append(lat)
                        lons.append(lon)
                        elevs.append(elev)
                        station_names.append(station_name)
                        count_ne=count_ne+1
    print(count_ne)          
    sys.exit()
    dk=xr.Dataset()
    
    dk['stations']=(('num'),stations)
    dk['lats']=(('num'),lats)
    dk['lons']=(('num'),lons)
    dk['elevs']=(('num'),elevs)
    dk['thresholds_winter_99']=(('num'),thresholds_winter_99)
    dk['thresholds_spring_99']=(('num'),thresholds_spring_99)
    dk['thresholds_summer_99']=(('num'),thresholds_summer_99)
    dk['thresholds_fall_99']=(('num'),thresholds_fall_99)
    
    dk['thresholds_winter_95']=(('num'),thresholds_winter_95)
    dk['thresholds_spring_95']=(('num'),thresholds_spring_95)
    dk['thresholds_summer_95']=(('num'),thresholds_summer_95)
    dk['thresholds_fall_95']=(('num'),thresholds_fall_95)
    
    dk['thresholds_annual_99']=(('num'),thresholds_annual_99)
    dk['thresholds_annual_95']=(('num'),thresholds_annual_95)
    dk['station_names']=(('num'),station_names)
    dk.coords['num']=np.arange(len(stations))
    print(station_names)
    print(dk['station_names'])
    try:
        os.remove(dir+'station_numbers_95.nc')#save eligible stations and corresponding thresholds, lats, lons in this file
    except OSError:
        pass
    dk.to_netcdf(dir+'station_numbers_95.nc',mode='w',format='NETCDF4',unlimited_dims='num')

                                                        0
0       ACW00011604  17.1167  -61.7833   10.1    ST JO...
1       ACW00011647  17.1333  -61.7833   19.2    ST JO...
2       AE000041196  25.3330   55.5170   34.0    SHARJ...
3       AEM00041194  25.2550   55.3640   10.4    DUBAI...
4       AEM00041217  24.4330   54.6510   26.8    ABU D...
...                                                   ...
118487  ZI000067969 -21.0500   29.3670  861.0    WEST ...
118488  ZI000067975 -20.0670   30.8670 1095.0    MASVI...
118489  ZI000067977 -21.0170   31.5830  430.0    BUFFA...
118490  ZI000067983 -20.2000   32.6160 1132.0    CHIPI...
118491  ZI000067991 -22.2170   30.0000  457.0    BEITB...

[118492 rows x 1 columns]
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
2

SystemExit: 

/kt11/ktyle/anaconda3_2001/envs/jun20/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3425: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
if 1==1:#part 2: read the obs at eligible stations into a better format
    yr_start=1979
    yr_end=2019
    tuples=[]
    years=[]
    months=[]
    days1=[]
    lats1=[]
    lons1=[]
    elevs1=[]
    observations=[]
    dates=[]
    station_names_list=[]
    stations_list=[]
    dp=xr.open_dataset(dir+'station_numbers_95.nc')
    stations=dp.stations.values.tolist()
    lats=dp.lats.values.tolist()
    lons=dp.lons.values.tolist()
    elevs=dp.elevs.values.tolist()
    station_names=dp.station_names.values.tolist()
    
    points=zip(lons,lats)
    for i in range(len(stations)):
        print(str(i+1)+'/'+str(len(stations)))
        lat=lats[i]
        lon=lons[i]
        elev=elevs[i]
        station_name=station_names[i]
        station=stations[i]
        filename=stations[i]
        ds1=pd.read_csv(filename,header=None)
        nancount=0
        qcount=0
        yescount=0
        for row1 in ds1.iterrows():
            row_neg_space=row1[1][0].replace('-',' -')
            row_basic=row1[1][0]
            a=row_neg_space.split()
            code=a[0][-4:]
            if code=='PRCP':
                year=int(a[0][11:15])
                if yr_start<=year<=yr_end:
                    month=int(a[0][15:17])
                    if month in [1,3,5,7,8,10,12]:
                        days=31
                    if month in [4,6,9,11]:
                        days=30
                    if month==2 and year%4==0:
                        days=29
                    if month==2 and year%4!=0:
                        days=28

                    for i in range(days):
                        #print(row_basic)
                        mflag=row_basic[26+8*i]
                        qflag=row_basic[27+8*i]
                        sflag=row_basic[28+8*i]
                        obs=float(row_basic[21+8*i:26+8*i])
                    
                        if obs==-9999.0:
                            pass
                        elif qflag!=' ':
                            pass
                        else:
                            #if 1980<=year<=2018 and month in [1,2,12] or year==1979 and month==12 or year==2019 and month in [1,2]:#winter
                            #if month in [9,10,11]:#fall
                            #if month in [6,7,8]:#summer
                            #if month in [3,4,5]:#spring
                            if 1==1:
                                years.append(year)
                                months.append(month)
                                days1.append(i)
                                observations.append(obs/10.)#convert from tenths of mm to mm
                                lats1.append(lat)
                                lons1.append(lon)
                                elevs1.append(elev)
                                date=dt.datetime(year,month,i+1,6)
                                dates.append(date)
                                station_names_list.append(station_name)
                                stations_list.append(station)

1/314
2/314
3/314
4/314
5/314
6/314
7/314
8/314
9/314
10/314
11/314
12/314
13/314
14/314
15/314
16/314
17/314
18/314
19/314
20/314
21/314
22/314
23/314
24/314
25/314
26/314
27/314
28/314
29/314
30/314
31/314
32/314
33/314
34/314
35/314
36/314
37/314
38/314
39/314
40/314
41/314
42/314
43/314
44/314
45/314
46/314
47/314
48/314
49/314
50/314
51/314
52/314
53/314
54/314
55/314
56/314
57/314
58/314
59/314
60/314
61/314
62/314
63/314
64/314
65/314
66/314
67/314
68/314
69/314
70/314
71/314
72/314
73/314
74/314
75/314
76/314
77/314
78/314
79/314
80/314
81/314
82/314
83/314
84/314
85/314
86/314
87/314
88/314
89/314
90/314
91/314
92/314
93/314
94/314
95/314
96/314
97/314
98/314
99/314
100/314
101/314
102/314
103/314
104/314
105/314
106/314
107/314
108/314
109/314
110/314
111/314
112/314
113/314
114/314
115/314
116/314
117/314
118/314
119/314
120/314
121/314
122/314
123/314
124/314
125/314
126/314
127/314
128/314
129/314
130/314
131/314
132/314
133/314
134/314
135/314
136/314
137/314
138/314
139/

In [3]:
print(thresholds_fall_99)#just checking threshold values
print(thresholds_annual_99)
print(thresholds_summer_99)

NameError: name 'thresholds_fall_99' is not defined

In [7]:
#for a range of station extreme and area sum thresholds: calculate extreme precip sums per day, then
#choose the top 5, 10, or 20% of extreme-precip-having days by the extreme precip sum.  These are 
#the "EP days".  

#ds.close()
yrs_neusa=np.arange(1979,2020,1)
ds=xr.open_dataset(dir+'station_numbers_95.nc')
station_urls=ds['stations'] 
print(len(station_urls.values))

thresholds_fall_99=ds['thresholds_fall_99']/10.
thresholds_spring_99=ds['thresholds_spring_99']/10.
thresholds_summer_99=ds['thresholds_summer_99']/10.
thresholds_winter_99=ds['thresholds_winter_99']/10.
thresholds_fall_95=ds['thresholds_fall_95']/10.
thresholds_spring_95=ds['thresholds_spring_95']/10.
thresholds_summer_95=ds['thresholds_summer_95']/10.
thresholds_winter_95=ds['thresholds_winter_95']/10.
thresholds_annual_99=ds['thresholds_annual_99']/10.
thresholds_annual_95=ds['thresholds_annual_95']/10.

print(thresholds_summer_99)
print(thresholds_winter_99)

seasons_list=['winter','winter','spring','spring','summer','summer','fall','fall','annual','annual']

station_thresholds_list=[thresholds_winter_99,thresholds_winter_95,thresholds_spring_99,
                         thresholds_spring_95,thresholds_summer_99,thresholds_summer_95,
                         thresholds_fall_99,thresholds_fall_95,thresholds_annual_99,thresholds_annual_95]

print(len(station_thresholds_list))
area_thresholds_list=[80,90,95]
thresholds_list=[99,95,99,95,99,95,99,95,99,95]
zipped=list(zip(dates,lats1,lons1,observations,stations_list))#stations_list is urls too
threshold_dict={}
        
for l1 in range(len(station_thresholds_list)):
    
    thresholds_choose=station_thresholds_list[l1]
    for i in range(len(station_urls)):
        threshold_dict[station_urls.values[i]]=thresholds_choose[i]
    season_choose=seasons_list[l1]
    thresh_choose=thresholds_list[l1]

    if season_choose=='winter':
        for i in range(len(yrs_neusa)):
            year=yrs_neusa[i]
            if year==1979:
                pass
            if 1980<=year<=2019:
                starting_date=dt.datetime(year-1,12,1,6)
                if year%4==0:
                    date_range=[starting_date+dt.timedelta(x) for x in range(91)]
                else:
                    date_range=[starting_date+dt.timedelta(x) for x in range(90)]
                if i==1:
                    all_dates=date_range
                else:
                    all_dates=all_dates+date_range
    else:
        for i in range(len(yrs_neusa)):
            year=yrs_neusa[i]
            if season_choose=='fall':
                starting_date=dt.datetime(year,9,1,6)
                date_range=[starting_date+dt.timedelta(x) for x in range(91)]
            if season_choose=='spring':
                starting_date=dt.datetime(year,3,1,6)
                date_range=[starting_date+dt.timedelta(x) for x in range(92)]
            if season_choose=='summer':
                starting_date=dt.datetime(year,6,1,6)
                date_range=[starting_date+dt.timedelta(x) for x in range(92)]
            if season_choose=='annual':
                starting_date=dt.datetime(year,9,1,6)
                date_range=[starting_date+dt.timedelta(x) for x in range(91)]
            if i==0:
                all_dates=date_range
            else:
                all_dates=all_dates+date_range
    #print(all_dates)
    daily_extreme_totals=[]
    for i in range(len(all_dates)):#now find extreme sums for each day
        if i%100==0:
            print(i)
        select_day=[x for x in zipped if x[0]==all_dates[i]]
        daily_extreme_sum=0
        count_day_values=0#how many valid observations are there?
        for j in range(len(select_day)):
            station_url=select_day[j][-1]
            obs=select_day[j][-2]

            if obs>=0:
                count_day_values=count_day_values+1
            threshold=threshold_dict[station_url]
            if obs>=threshold:#add extreme values
                daily_extreme_sum=daily_extreme_sum+obs
            if j==len(select_day)-1:
                if count_day_values/float(len(station_urls))>=0.90:#only count days with 90%+ data coverage (should be almost all days)
                    daily_extreme_totals.append(float(daily_extreme_sum)/float(count_day_values))
                else:
                    daily_extreme_totals.append(np.nan)
    
    for l2 in range(len(area_thresholds_list)):
        
        area_threshold_choose=area_thresholds_list[l2]
        
        print(thresholds_choose)
        print(area_threshold_choose)
        print(season_choose)
        print(thresh_choose)
        
        print('MAX THRESH')
        print(max(thresholds_choose).values)
        print(len(all_dates))
        print(len(daily_extreme_totals))
        print(len([x for x in daily_extreme_totals if x>0]))
        print(float(sum([x for x in daily_extreme_totals if x>0]))/float(len([x for x in daily_extreme_totals if x>0])))
                        
        zipped_sums=list(zip(all_dates,daily_extreme_totals))
        nonzero_extremes=[x for x in daily_extreme_totals if x>0]
        extreme_sum_threshold=np.percentile(nonzero_extremes,area_threshold_choose,interpolation='linear')
        zipped_select=[x for x in zipped_sums if x[1]>=extreme_sum_threshold]
        dates_extreme=[x[0] for x in zipped_select]
        print(len(dates_extreme))
        data_select=[x for x in zipped if x[0] in dates_extreme]
        dates_select=[x[0] for x in data_select]
        lats_select=[x[1] for x in data_select]
        lons_select=[x[2] for x in data_select]
        obs_select=[x[3] for x in data_select]  
        stations_select=[x[4] for x in data_select]
        
        dw=xr.Dataset()
        dw['lats']=(('station'),lats_select)
        dw['lons']=(('station'),lons_select)
        dw['obs']=(('station'),obs_select)
        dw['dates']=(('station'),dates_select)
        dw['stations']=(('station'),stations_select)
        dw.coords['station']=np.arange(len(lats_select))
        
        #save set of extreme days with corresponding filename
        try:
            os.remove(dir+'extreme_days_ghcnd_'+str(thresh_choose)+'_'+str(area_threshold_choose)+'_'+season_choose+'.nc')
            #os.remove(dir+'extreme_days_ghcnd_fall_99_80.nc')
        except OSError:
            pass
        #dw.to_netcdf(dir+'extreme_days_ghcnd_fall_99_80.nc',mode='w',format='NETCDF4',unlimited_dims='station')
        dw.to_netcdf(dir+'extreme_days_ghcnd_'+str(thresh_choose)+'_'+str(area_threshold_choose)+'_'+season_choose+'.nc',mode='w',format='NETCDF4',unlimited_dims='station')

314
<xarray.DataArray 'thresholds_summer_99' (num: 314)>
array([66.445, 75.545, 64.28 , 81.8  , 87.655, 90.549, 55.48 , 47.84 ,
       49.8  , 49.43 , 50.3  , 57.765, 46.5  , 61.   , 48.8  , 46.194,
       45.84 , 47.584, 47.08 , 43.48 , 62.42 , 42.024, 67.912, 47.015,
       55.9  , 59.875, 65.525, 49.5  , 75.475, 48.024, 67.1  , 59.04 ,
       55.341, 68.04 , 56.71 , 83.134, 58.415, 57.875, 64.3  , 71.034,
       74.924, 65.775, 62.931, 68.022, 73.123, 61.   , 67.8  , 63.89 ,
       59.45 , 72.82 , 64.537, 63.5  , 80.08 , 63.008, 64.376, 45.53 ,
       56.816, 59.535, 43.372, 58.92 , 54.2  , 58.9  , 47.305, 54.073,
       54.305, 61.   , 65.65 , 66.747, 62.78 , 75.08 , 64.738, 61.086,
       69.095, 69.979, 61.825, 69.138, 46.18 , 50.795, 48.69 , 41.59 ,
       44.755, 44.454, 56.614, 42.292, 49.   , 50.8  , 47.739, 40.6  ,
       51.8  , 45.602, 43.876, 48.41 , 39.9  , 48.8  , 48.416, 44.668,
       48.066, 51.878, 52.875, 63.557, 48.95 , 47.488, 50.554, 49.659,
       47.27 , 82.19

<xarray.DataArray 'thresholds_winter_99' (num: 314)>
array([50.305, 53.685, 47.166, 61.138, 47.856, 47.025, 58.346, 33.081,
       39.319, 40.9  , 42.174, 47.734, 31.8  , 51.77 , 53.34 , 42.55 ,
       33.   , 30.955, 41.882, 35.905, 47.2  , 36.202, 52.632, 36.537,
       40.1  , 61.   , 40.962, 39.29 , 46.38 , 37.1  , 52.118, 42.7  ,
       34.92 , 49.739, 39.96 , 48.523, 48.08 , 43.17 , 45.665, 56.1  ,
       62.13 , 53.61 , 50.136, 48.925, 54.616, 51.365, 56.914, 51.645,
       52.4  , 50.3  , 65.22 , 50.8  , 53.483, 49.474, 53.98 , 41.094,
       47.36 , 45.148, 24.4  , 51.08 , 43.435, 41.472, 27.056, 50.8  ,
       37.185, 56.91 , 62.799, 45.574, 50.8  , 38.81 , 55.084, 47.164,
       41.815, 50.084, 48.16 , 47.179, 30.901, 30.63 , 25.675, 23.6  ,
       26.788, 30.76 , 40.6  , 25.4  , 30.5  , 33.628, 30.69 , 24.442,
       28.748, 23.088, 25.92 , 27.305, 23.609, 38.824, 24.064, 32.91 ,
       23.836, 29.124, 32.806, 48.032, 33.   , 37.27 , 34.405, 24.382,
       27.7  , 53.613, 2

<xarray.DataArray 'thresholds_winter_95' (num: 314)>
array([32.8  , 35.055, 28.955, 36.3  , 30.5  , 33.   , 35.6  , 22.47 ,
       26.7  , 27.4  , 26.73 , 30.   , 20.15 , 32.29 , 29.5  , 25.4  ,
       18.   , 19.875, 25.4  , 21.025, 30.73 , 20.1  , 33.06 , 20.6  ,
       25.445, 41.49 , 27.9  , 22.96 , 29.2  , 22.84 , 30.315, 28.825,
       20.95 , 29.2  , 22.4  , 31.64 , 27.9  , 29.2  , 26.83 , 34.3  ,
       34.3  , 31.71 , 32.895, 30.   , 35.6  , 32.43 , 33.5  , 32.5  ,
       32.9  , 32.115, 36.85 , 33.045, 33.   , 30.375, 32.   , 26.52 ,
       27.8  , 28.11 , 14.85 , 33.3  , 25.4  , 26.22 , 15.7  , 31.8  ,
       24.1  , 31.8  , 31.8  , 31.8  , 35.175, 26.21 , 33.6  , 33.32 ,
       28.275, 33.58 , 30.5  , 30.455, 16.105, 17.   , 16.225, 14.7  ,
       14.5  , 16.   , 22.9  , 13.2  , 19.1  , 19.76 , 20.3  , 14.   ,
       18.24 , 13.64 , 15.2  , 16.5  , 13.5  , 22.72 , 14.   , 20.62 ,
       14.12 , 16.53 , 18.8  , 30.5  , 19.1  , 25.4  , 18.515, 13.41 ,
       16.8  , 33.635, 1

<xarray.DataArray 'thresholds_spring_99' (num: 314)>
array([61.52 , 65.968, 56.072, 79.288, 55.464, 55.855, 65.57 , 37.702,
       36.139, 42.415, 44.404, 52.77 , 30.665, 54.6  , 47.435, 45.032,
       33.57 , 35.1  , 44.544, 36.08 , 53.055, 39.824, 66.76 , 33.297,
       45.7  , 73.7  , 47.5  , 41.4  , 50.97 , 44.134, 51.685, 52.827,
       42.004, 59.87 , 40.544, 56.8  , 51.548, 59.16 , 53.483, 67.22 ,
       63.136, 52.387, 63.62 , 55.277, 65.16 , 57.064, 70.158, 68.57 ,
       60.982, 61.292, 72.165, 54.46 , 66.   , 53.778, 63.97 , 46.7  ,
       49.085, 56.019, 30.892, 67.915, 45.   , 43.395, 35.37 , 55.996,
       43.265, 57.608, 60.2  , 54.349, 58.246, 53.392, 77.139, 61.717,
       57.402, 59.422, 53.3  , 55.4  , 36.61 , 36.929, 32.306, 33.488,
       34.472, 34.512, 46.962, 32.83 , 39.26 , 41.09 , 35.615, 30.5  ,
       35.6  , 32.   , 33.623, 32.725, 32.155, 39.96 , 31.08 , 39.27 ,
       33.86 , 30.952, 33.5  , 57.4  , 38.675, 46.7  , 39.304, 33.573,
       33.916, 66.89 , 2

<xarray.DataArray 'thresholds_spring_95' (num: 314)>
array([33.56 , 36.8  , 29.985, 42.7  , 34.42 , 35.73 , 36.95 , 23.51 ,
       25.655, 27.73 , 25.5  , 30.955, 20.23 , 33.   , 29.15 , 26.9  ,
       21.17 , 21.3  , 27.9  , 22.4  , 32.   , 21.68 , 38.1  , 20.6  ,
       29.2  , 40.475, 31.2  , 27.3  , 31.35 , 28.4  , 33.   , 30.7  ,
       24.4  , 33.75 , 26.7  , 32.24 , 32.41 , 32.24 , 31.   , 34.8  ,
       34.92 , 34.31 , 37.525, 35.275, 35.36 , 31.8  , 36.1  , 32.8  ,
       33.   , 33.92 , 37.82 , 32.44 , 36.8  , 32.3  , 33.87 , 26.88 ,
       29.55 , 33.   , 20.1  , 35.6  , 27.3  , 26.9  , 20.1  , 34.   ,
       24.9  , 31.   , 33.8  , 35.13 , 36.8  , 31.2  , 38.1  , 35.375,
       31.8  , 37.1  , 33.5  , 34.5  , 20.83 , 22.9  , 20.63 , 19.8  ,
       20.6  , 21.36 , 25.99 , 20.3  , 24.9  , 25.49 , 22.1  , 18.5  ,
       21.7  , 20.17 , 19.315, 19.8  , 19.6  , 25.55 , 20.1  , 24.54 ,
       21.1  , 20.3  , 20.75 , 36.1  , 22.95 , 29.2  , 23.72 , 20.3  ,
       20.8  , 35.435, 1

...

<xarray.DataArray 'thresholds_summer_95' (num: 314)>
array([35.705, 40.735, 35.79 , 42.2  , 43.31 , 45.19 , 35.69 , 26.7  ,
       30.2  , 29.2  , 27.7  , 31.5  , 25.9  , 30.86 , 29.91 , 28.17 ,
       28.2  , 26.7  , 28.85 , 26.7  , 31.5  , 25.595, 34.71 , 27.4  ,
       27.9  , 32.35 , 36.8  , 31.53 , 40.6  , 30.2  , 39.4  , 35.86 ,
       31.8  , 38.6  , 30.   , 43.23 , 38.1  , 34.95 , 38.505, 36.1  ,
       36.3  , 37.5  , 36.6  , 35.8  , 38.9  , 34.3  , 40.04 , 37.95 ,
       35.6  , 38.1  , 36.8  , 37.1  , 41.66 , 40.03 , 36.33 , 28.095,
       34.18 , 34.3  , 29.79 , 37.64 , 35.1  , 33.3  , 27.9  , 35.6  ,
       31.   , 32.5  , 33.3  , 39.4  , 38.9  , 37.26 , 38.6  , 39.45 ,
       39.385, 50.075, 40.675, 41.09 , 25.7  , 34.275, 29.335, 24.6  ,
       25.565, 27.9  , 33.5  , 26.99 , 30.7  , 31.2  , 28.125, 27.9  ,
       29.6  , 27.975, 26.4  , 28.7  , 26.36 , 33.3  , 29.   , 26.7  ,
       26.9  , 29.7  , 29.425, 40.1  , 28.4  , 30.5  , 29.   , 30.395,
       26.7  , 39.9  , 2

<xarray.DataArray 'thresholds_fall_99' (num: 314)>
array([ 70.085,  84.712,  68.04 ,  77.645,  67.45 ,  71.61 ,  79.092,
        49.94 ,  50.71 ,  53.702,  56.375,  63.55 ,  48.3  ,  69.195,
        63.71 ,  54.4  ,  50.36 ,  44.836,  58.175,  49.   ,  71.164,
        46.335,  72.756,  47.416,  57.044,  85.86 ,  70.42 ,  54.125,
        71.005,  57.706,  86.6  ,  68.97 ,  43.702,  65.55 ,  52.82 ,
        94.036,  56.824,  71.964,  61.614,  76.41 ,  61.69 ,  64.138,
        64.386,  63.726,  69.42 ,  74.61 ,  71.199,  71.998,  76.608,
        70.275,  69.44 ,  71.612,  66.104,  60.164,  64.408,  67.425,
        70.422,  68.48 ,  42.4  ,  73.704,  54.9  ,  64.824,  43.91 ,
        66.772,  52.234,  63.5  ,  82.742,  71.276,  75.063,  59.44 ,
        88.265,  75.4  ,  60.404,  73.3  ,  76.872,  64.21 ,  47.175,
        43.232,  44.125,  44.5  ,  39.307,  44.432,  57.8  ,  42.134,
        62.77 ,  49.72 ,  49.9  ,  41.928,  52.225,  41.545,  43.9  ,
        52.24 ,  41.795,  57.915,  46.4

<xarray.DataArray 'thresholds_fall_95' (num: 314)>
array([41.9  , 47.24 , 37.3  , 44.2  , 39.9  , 40.95 , 45.425, 27.9  ,
       29.7  , 33.59 , 34.35 , 37.15 , 25.22 , 41.05 , 36.1  , 33.3  ,
       29.8  , 25.9  , 34.45 , 26.9  , 40.72 , 25.875, 43.08 , 24.9  ,
       34.3  , 47.68 , 35.965, 30.5  , 39.615, 30.245, 42.2  , 36.905,
       25.4  , 42.05 , 29.   , 43.2  , 40.6  , 37.6  , 36.1  , 40.9  ,
       41.625, 38.1  , 43.275, 39.6  , 40.4  , 38.91 , 41.9  , 36.435,
       40.11 , 38.65 , 43.2  , 37.78 , 44.5  , 40.3  , 38.75 , 31.85 ,
       40.49 , 37.04 , 23.44 , 43.2  , 34.3  , 33.42 , 25.9  , 37.3  ,
       31.   , 37.475, 47.45 , 39.63 , 45.11 , 39.35 , 48.   , 43.2  ,
       36.18 , 40.525, 39.8  , 39.9  , 23.05 , 24.56 , 23.6  , 26.2  ,
       22.235, 24.08 , 32.3  , 21.8  , 29.2  , 28.4  , 25.4  , 24.1  ,
       26.625, 23.9  , 22.4  , 26.35 , 21.6  , 32.5  , 23.9  , 26.2  ,
       24.51 , 26.82 , 24.9  , 47.   , 26.4  , 30.5  , 27.2  , 22.9  ,
       25.77 , 39.59 , 24.

<xarray.DataArray 'thresholds_annual_99' (num: 314)>
array([62.78 , 72.99 , 59.662, 74.53 , 67.453, 64.422, 64.636, 43.4  ,
       45.2  , 48.311, 49.5  , 57.487, 40.6  , 60.883, 53.104, 46.895,
       43.9  , 39.9  , 48.   , 41.7  , 58.84 , 40.968, 67.856, 41.547,
       50.8  , 71.4  , 56.6  , 44.7  , 57.648, 47.11 , 63.388, 56.106,
       44.765, 61.47 , 46.2  , 68.6  , 55.06 , 57.4  , 57.632, 67.294,
       64.231, 61.2  , 62.7  , 57.254, 66.602, 61.94 , 67.285, 67.387,
       59.7  , 65.3  , 68.365, 60.2  , 68.85 , 56.115, 63.2  , 49.56 ,
       56.896, 59.064, 37.724, 66.5  , 51.3  , 52.065, 38.307, 55.9  ,
       45.7  , 60.404, 72.516, 59.7  , 62.773, 58.43 , 71.085, 62.862,
       59.765, 65.267, 58.4  , 59.9  , 40.385, 43.2  , 40.891, 37.6  ,
       37.228, 40.6  , 49.   , 37.42 , 46.82 , 44.5  , 42.915, 37.8  ,
       45.2  , 37.6  , 37.043, 42.014, 36.125, 46.449, 40.846, 42.318,
       39.1  , 42.173, 42.068, 63.952, 43.   , 48.62 , 44.2  , 40.322,
       38.926, 65.95 , 3

<xarray.DataArray 'thresholds_annual_95' (num: 314)>
array([36.8  , 39.6  , 33.09 , 41.9  , 37.3  , 38.6  , 38.9  , 25.4  ,
       28.24 , 29.2  , 29.12 , 32.5  , 23.02 , 34.665, 31.5  , 28.7  ,
       24.145, 23.6  , 29.2  , 24.1  , 33.5  , 23.9  , 37.8  , 23.6  ,
       29.23 , 41.235, 32.5  , 27.9  , 35.8  , 27.35 , 36.3  , 33.03 ,
       25.7  , 36.1  , 26.9  , 37.6  , 35.6  , 33.8  , 33.5  , 36.3  ,
       36.8  , 35.3  , 37.7  , 35.1  , 38.1  , 34.5  , 37.6  , 35.1  ,
       35.1  , 36.1  , 38.795, 34.71 , 39.1  , 36.375, 35.65 , 27.9  ,
       32.03 , 33.8  , 22.4  , 37.6  , 31.05 , 30.13 , 22.4  , 35.6  ,
       28.4  , 33.62 , 37.1  , 36.8  , 39.4  , 32.59 , 39.57 , 38.1  ,
       34.3  , 39.9  , 36.3  , 37.1  , 21.8  , 24.6  , 22.6  , 21.6  ,
       21.1  , 23.4  , 28.4  , 21.1  , 26.46 , 26.7  , 23.9  , 21.3  ,
       24.6  , 22.1  , 20.8  , 22.9  , 20.8  , 28.2  , 21.3  , 24.6  ,
       22.1  , 23.4  , 22.9  , 38.34 , 24.6  , 28.7  , 24.48 , 22.01 ,
       22.1  , 37.6  , 2